21.03.23 ML_DL Tensorflow Ver.1 사용해서 분석하기

# Tensorflow_Ver.1에서의 ML_DL

## multinomial classification(3개이상 그룹)

In [12]:
# training data set(교안 pt.54)
x_data = [[10,7,8,5],
          [8,8,9,4],
          [7,8,2,3],
          [6,3,9,3],
          [7,5,7,4],
          [3,5,6,2],
          [2,4,3,1]]
# 종속변수는 multinomial classification에서는 원핫인코딩(교안pt.58)
y_data =[[1, 0, 0],
         [1, 0, 0],
         [0, 1, 0],
         [0, 1, 0],
         [0, 1, 0],
         [0, 0, 1],
         [0, 0, 1]]
# placeholder
X = tf.placeholder(shape=[None, 4], dtype=tf.float32)
Y = tf.placeholder(shape=[None, 3], dtype=tf.float32)

# Weight(4x3행렬) , bias(3개)
W = tf.Variable(tf.random_normal([4,3]), name="weight")
b = tf.Variable(tf.random_normal([3])  , name="bias")

# Hypothesis
# logits = X @ W + b
logits = tf.matmul(X, W) + b
# H = tf.nn.sigmoid(logits)
H = tf.nn.softmax(logits) # softmax분류분석 최종단계에서 결과의 합이 1이 되도록

# cost function
# cost = tf.reduce_mean(tf.square(H - Y))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                        labels=Y))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(1, 6001):
    _, cost_val = sess.run([train, cost], feed_dict={X:x_data, Y:y_data})
    if step%300 == 0:
        print("{}번째 cost : {}".format(step, cost_val))

300번째 cost : 2.2000772953033447
600번째 cost : 1.6827433109283447
900번째 cost : 1.3214807510375977
1200번째 cost : 1.0687787532806396
1500번째 cost : 0.8274825811386108
1800번째 cost : 0.5589641332626343
2100번째 cost : 0.051173750311136246
2400번째 cost : 0.046326618641614914
2700번째 cost : 0.042815469205379486
3000번째 cost : 0.040038976818323135
3300번째 cost : 0.037729229778051376
3600번째 cost : 0.03574393317103386
3900번째 cost : 0.03399952873587608
4200번째 cost : 0.0324433334171772
4500번째 cost : 0.03103957138955593
4800번째 cost : 0.02976263128221035
5100번째 cost : 0.028593355789780617
5400번째 cost : 0.027517128735780716
5700번째 cost : 0.026521963998675346
6000번째 cost : 0.025598425418138504


In [13]:
# 예측
result = sess.run(H, feed_dict={X:[[8,8,9,4]]})
print(result)
print(result.argmax(axis=1)) # 0번째 열만 1

[[0.97757614 0.01979787 0.00262603]]
[0]


In [14]:
# accuracy 측정.
# H => 0.97 0.19 0.002을 경우 H.argmax => 0
# Y -> 1    0    0    을 경우 Y.argmax => 0
predict = tf.argmax(H, axis=1) # 어떤 열의 값이 제일 큰지 index반환
correct = tf.equal(predict, tf.argmax(Y, 1))
# print(sess.run(correct, feed_dict={X:x_data, Y:y_data}))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print(sess.run(accuracy, feed_dict={X:x_data, Y:y_data})*100, '%')

100.0 %


## XOR(교안 p77)

In [19]:
# training data set
x_data = [[0,0], [0,1], [1,0], [1,1]]
y_data = [[0],   [1],   [1],   [0]]
# placeholder
X = tf.placeholder(shape=[None, 2], dtype=tf.float32)
Y = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random_normal([2,1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")
# Hypothesis
# H = X@W + b
logits = tf.matmul(X, W) + b
H = tf.sigmoid(logits)

# cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                             labels=Y))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 3001):
    _, cost_val = sess.run([train, cost], feed_dict={X:x_data, Y:y_data})
    if step%300 == 0:
        print("{}번째 cost : {}".format(step, cost_val))
# accuracy 측정
predict = tf.cast(H>0.5, dtype=tf.float32)
correct = tf.equal(predict, Y) # False True False True
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print('정확도 :', sess.run(accuracy, feed_dict={X:x_data, Y:y_data}))  

300번째 cost : 0.7366209626197815
600번째 cost : 0.7234445810317993
900번째 cost : 0.7145653963088989
1200번째 cost : 0.708294153213501
1500번째 cost : 0.7038532495498657
1800번째 cost : 0.7007167935371399
2100번째 cost : 0.6985057592391968
2400번째 cost : 0.6969480514526367
2700번째 cost : 0.6958501935005188
3000번째 cost : 0.6950753927230835
정확도 : 0.5


In [20]:
# predict
predict = tf.cast(H>0.5, dtype=tf.float32)
print(sess.run(predict, feed_dict={X:x_data} ) )

[[0.]
 [0.]
 [1.]
 [1.]]


## Deep Learning XOR 예제

In [21]:
# training data set
x_data = [[0,0], [0,1], [1,0], [1,1]]
y_data = [[0],   [1],   [1],   [0]]
# placeholder
X = tf.placeholder(shape=[None, 2], dtype=tf.float32)
Y = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# layer 추가
# Weight & bias(layer1= 입력2개 출력 4)
W1 = tf.Variable(tf.random_normal([2,4]), name="weight1")
b1 = tf.Variable(tf.random_normal([4]), name="bias1")
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)
# Weight & bias(layer2 = 입력4개, 출력1개)
W2 = tf.Variable(tf.random_normal([4,1]), name="weight2")
b2 = tf.Variable(tf.random_normal([1]), name="bias2")

# Hypothesis
logits = tf.matmul(layer1, W2) + b2
H = tf.sigmoid(logits)

# cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                             labels=Y))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(1, 3001):
    _, cost_val = sess.run([train, cost], feed_dict={X:x_data, Y:y_data})
    if step%300 == 0:
        print("{}번째 cost : {}".format(step, cost_val))

# accuracy 측정
predict = tf.cast(H>0.5, dtype=tf.float32)
correct = tf.equal(predict, Y) # False True False True
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print('정확도 :', sess.run(accuracy, feed_dict={X:x_data, Y:y_data}))

300번째 cost : 0.756483256816864
600번째 cost : 0.7134456634521484
900번째 cost : 0.7041211128234863
1200번째 cost : 0.6970817446708679
1500번째 cost : 0.6912665367126465
1800번째 cost : 0.6862974762916565
2100번째 cost : 0.6818861365318298
2400번째 cost : 0.6778103113174438
2700번째 cost : 0.6739010810852051
3000번째 cost : 0.670031726360321
정확도 : 0.5


In [23]:
# training data set
x_data = [[0,0], [0,1], [1,0], [1,1]]
y_data = [[0],   [1],   [1],   [0]]
# placeholder
X = tf.placeholder(shape=[None, 2], dtype=tf.float32)
Y = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# layer 추가
# Weight & bias(layer1= 입력2개 출력 10)
W1 = tf.Variable(tf.random_normal([2,10]), name="weight1")
b1 = tf.Variable(tf.random_normal([10]), name="bias1")
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)
# Weight & bias(layer2 = 입력10개, 출력1개)
W2 = tf.Variable(tf.random_normal([10,20]), name="weight2")
b2 = tf.Variable(tf.random_normal([20]), name="bias2")
layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)
# Weight & bias(layer3 = 입력20개, 출력10개)
W3 = tf.Variable(tf.random_normal([20,10]), name="weight3")
b3 = tf.Variable(tf.random_normal([10]), name="bias3")
layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)
# Weight & bias(output layer = 입력10개, 출력1개)
W4 = tf.Variable(tf.random_normal([10,1]), name="weight4")
b4 = tf.Variable(tf.random_normal([1]), name="bias4")

# Hypothesis
logits = tf.matmul(layer3, W4) + b4
H = tf.sigmoid(logits)

# cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                             labels=Y))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(1, 3001):
    _, cost_val = sess.run([train, cost], feed_dict={X:x_data, Y:y_data})
    if step%300 == 0:
        print("{}번째 cost : {}".format(step, cost_val))

# accuracy 측정
predict = tf.cast(H>0.5, dtype=tf.float32)
correct = tf.equal(predict, Y) # False True False True
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print('정확도 :', sess.run(accuracy, feed_dict={X:x_data, Y:y_data}))

300번째 cost : 0.6979430317878723
600번째 cost : 0.6919685006141663
900번째 cost : 0.6865856647491455
1200번째 cost : 0.6815888285636902
1500번째 cost : 0.6768162250518799
1800번째 cost : 0.6721378564834595
2100번째 cost : 0.6674453020095825
2400번째 cost : 0.6626453995704651
2700번째 cost : 0.6576552987098694
3000번째 cost : 0.6523995399475098
정확도 : 0.75


In [24]:
# training data set
x_data = [[0,0], [0,1], [1,0], [1,1]]
y_data = [[0],   [1],   [1],   [0]]
# placeholder
X = tf.placeholder(shape=[None, 2], dtype=tf.float32)
Y = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# layer 추가
# Weight & bias(layer1= 입력2개 출력 10)
W1 = tf.Variable(tf.random_normal([2,10]), name="weight1")
b1 = tf.Variable(tf.random_normal([10]), name="bias1")
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)

# Weight & bias(layer2 = 입력10개, 출력20개)
W2 = tf.Variable(tf.random_normal([10,20]), name="weight2")
b2 = tf.Variable(tf.random_normal([20]), name="bias2")
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)

# Weight & bias(layer3 = 입력20개, 출력 10)
W3 = tf.Variable(tf.random_normal([20,10]), name="weight3")
b3 = tf.Variable(tf.random_normal([10]), name="bias3")
layer3 = tf.nn.relu(tf.matmul(layer2, W3)+b3)

# Weight & bias(output layer = 입력10개, 출력1)
W4 = tf.Variable(tf.random_normal([10,1]), name="weight4")
b4 = tf.Variable(tf.random_normal([1]), name="bias4")

# Hypothesis
logits = tf.matmul(layer3, W4) + b4
H = tf.sigmoid(logits)

# cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                             labels=Y))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 3001):
    _, cost_val = sess.run([train, cost], feed_dict={X:x_data, Y:y_data})
    if step%300 == 0:
        print("{}번째 cost : {}".format(step, cost_val))
# accuracy 측정
predict = tf.cast(H>0.5, dtype=tf.float32)
correct = tf.equal(predict, Y) # False True False True
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print('정확도 :', sess.run(accuracy, feed_dict={X:x_data, Y:y_data}))

300번째 cost : 0.014634678140282631
600번째 cost : 0.005764896050095558
900번째 cost : 0.003401009598746896
1200번째 cost : 0.002347290050238371
1500번째 cost : 0.001764932880178094
1800번째 cost : 0.0014005909906700253
2100번째 cost : 0.00115313904825598
2400번째 cost : 0.000976618961431086
2700번째 cost : 0.000845576636493206
3000번째 cost : 0.0007435418083332479
정확도 : 1.0
